In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from datetime import datetime, date
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import nevergrad as ng
import plotly.express as px

from estival.model import BayesianCompartmentalModel
from estival.optimization.nevergrad import optimize_model
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget, CustomTarget
from estival.calibration import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import *

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
time_start = 1800
time_end = 2020
time_step = 0.1

In [3]:
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

In [4]:
params = {
    "contact_rate": 0.009,
    "start_population_size": 200000,
    "progression_multiplier": 2,
    "seed": 1,
    "rr_infection_latent": 0.5,
    "rr_infection_recovered": 0.5,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.4,
}

In [5]:
tb_model, build_text = model.build_base_model(compartments, infectious_compartments, time_start, time_end, time_step)

In [6]:
model.set_starting_conditions(tb_model, params["start_population_size"], params["seed"])



'The simulation starts with 0.2 million fully susceptible persons, with infectious persons introduced later through strain seeding as described below. '

In [7]:
model.add_infection(tb_model)

('The infection process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_latent process moves people from the late_latent compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_recovered process moves people from the late_latent compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ')

In [8]:
model.add_latency(tb_model)

('The stabilisation process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The late_activation process moves people from the late_latent compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [9]:
model.add_treatment(tb_model)

('The treatment_recovery process moves people from the on_treatment compartment to the recovered compartment, under the frequency-dependent transmission assumption. ',
 'The treatment_death process moves people from the on_treatment compartment to the death, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the on_treatment compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [10]:
model.add_entry_flow(tb_model)

'The birth process add newborns to the model '

In [11]:
model.add_natural_death_flow(tb_model)

In [12]:
age_strata = [0,5,15,35,50]

In [13]:
matrix = model.build_contact_matrix(age_strata)

In [14]:
matrix

array([[ 398.43289672,  165.78966683,  231.75164317,  141.94492435,
          67.30073632],
       [ 261.82020387,  881.63067677,  311.38983781,  310.88835505,
         170.46333134],
       [ 643.68286218,  532.84120554,  915.52884268,  786.13676958,
         647.30153978],
       [ 401.62199159,  550.75979227,  673.30894113, 1134.31076003,
        1018.81243422],
       [ 356.13449939,  285.62836724,  664.14577066,  938.03403291,
        1763.57657715]])

In [15]:
#strat = model.add_age_strat(compartments, infectious_compartments, [0,5,15,35,50], matrix, fixed_parameters)

In [16]:
#tb_model.stratify_with(strat)

In [17]:
tb_model.run(parameters=params)

In [22]:
tb_model.get_epoch()